# Imports

In [ ]:
from datasets import load_dataset
import pandas as pd
import re

# Load and Preprocess

In [ ]:
ds = load_dataset("virattt/financial-qa-10K")

In [ ]:
ds

In [ ]:
ds = ds["train"].to_pandas()
ds

In [ ]:
ds = ds.rename(columns={
    "question": "query",
    "context": "corpus"
})


In [ ]:
df = pd.read_csv("TrainingData_V3.csv")
df

In [ ]:
df["corpus"]

In [ ]:
for idx, row in df.iterrows():
    row["corpus"] = re.sub(r"(\\u[0-9A-Fa-f]{4})+", " ", row["corpus"])

In [ ]:
subsets = df["subset"].unique().tolist()
eval_list=[]
train_list=[]
for subset in subsets:
    eval_list.append(df.loc[df["subset"]==subset][:15])
    train_list.append(df.loc[df["subset"]==subset][15:])

In [ ]:
eval_df = eval_list[0]
for i in range(1,len(eval_list)):
    eval_df = pd.concat([eval_df,eval_list[i]])

In [ ]:
train_df = train_list[0]
for i in range(1,len(train_list)):
    train_df = pd.concat([train_df,train_list[i]])

In [ ]:
eval_df = eval_df[["query", "corpus"]].copy()
train_df = train_df[["query", "corpus"]].copy()
ds_new = ds[["query", "corpus"]].copy()

In [ ]:
shuffled_df = ds_new.sample(frac=1).reset_index(drop=True)
eval_df = pd.concat([eval_df, shuffled_df[:800]])
train_df = pd.concat([train_df, shuffled_df[800:]])

In [ ]:
train_df = train_df.sample(frac=1).reset_index(drop=True)
eval_df = eval_df.sample(frac=1).reset_index(drop=True)

train_df.dropna(how='all', inplace=True)
eval_df.dropna(how='all', inplace=True)

train_df.dropna(subset=['query'], inplace=True)
eval_df.dropna(subset=['query'], inplace=True)

train_df.dropna(subset=['corpus'], inplace=True)
eval_df.dropna(subset=['corpus'], inplace=True)

# Save

In [ ]:
train_df.to_csv("train_df.csv",errors=False, header=True, index=False)
eval_df.to_csv("eval_df.csv",errors=False, header=True, index=False)